In [203]:
!pip install -q kagglehub ipywidgets plotly pandas

In [204]:
import plotly.io as pio
pio.renderers.default = "colab"

In [205]:
import kagglehub
path = kagglehub.dataset_download("rajanand/crime-in-india")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/crime-in-india


In [206]:
import kagglehub
import os
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

In [207]:
csv_files = sorted([f for f in os.listdir(path) if f.endswith(".csv")])
print(f"{len(csv_files)} CSV files found.")

file_dropdown = widgets.Dropdown(options=csv_files, description="Select CSV:")
display(file_dropdown)

19 CSV files found.


Dropdown(description='Select CSV:', options=('10_Property_stolen_and_recovered.csv', '20_Victims_of_rape.csv',…

In [208]:
def load_csv(file):
    df = pd.read_csv(os.path.join(path, file))
    return df

df = load_csv(file_dropdown.value)
df.head()

,Area_Name,Year,Group_Name,Sub_Group_Name,Cases_Property_Recovered,Cases_Property_Stolen,Value_of_Property_Recovered,Value_of_Property_Stolen
0,Andaman & Nicobar Islands,2001,Burglary - Property,3. Burglary,27,64,755858,1321961
1,Andhra Pradesh,2001,Burglary - Property,3. Burglary,3321,7134,51483437,147019348
2,Arunachal Pradesh,2001,Burglary - Property,3. Burglary,66,248,825115,4931904
3,Assam,2001,Burglary - Property,3. Burglary,539,2423,3722850,21466955
4,Bihar,2001,Burglary - Property,3. Burglary,367,3231,2327135,17023937


In [209]:
output = widgets.Output()

chart_dropdown = widgets.Dropdown(
    options=["Bar Chart", "Line Chart", "Pie Chart"],
    description="Chart Type:"
)

x_dropdown = widgets.Dropdown(description="X Axis:")
y_dropdown = widgets.Dropdown(description="Y Axis:")

def update_dropdowns(df):
    num_cols = df.select_dtypes(include='number').columns.tolist()
    cat_cols = df.select_dtypes(include='object').columns.tolist()
    x_dropdown.options = cat_cols + num_cols
    y_dropdown.options = num_cols

update_dropdowns(df)

display(chart_dropdown, x_dropdown, y_dropdown, output)

Dropdown(description='Chart Type:', options=('Bar Chart', 'Line Chart', 'Pie Chart'), value='Bar Chart')

Dropdown(description='X Axis:', options=('Area_Name', 'Group_Name', 'Sub_Group_Name', 'Year', 'Cases_Property_…

Dropdown(description='Y Axis:', options=('Year', 'Cases_Property_Recovered', 'Cases_Property_Stolen', 'Value_o…

Output()

In [213]:
def plot_graph(change=None):
    output.clear_output()
    with output:
        x = x_dropdown.value
        y = y_dropdown.value
        chart_type = chart_dropdown.value

        if x is None or y is None:
            print("Please select both X and Y axes.")
            return

        if chart_type == "Bar Chart":
            fig = px.bar(df, x=x, y=y, title=f"{y} by {x}")
        elif chart_type == "Line Chart":
            fig = px.line(df, x=x, y=y, title=f"{y} over {x}")
        elif chart_type == "Pie Chart":
            fig = px.pie(df, names=x, values=y, title=f"{y} distribution by {x}")
        else:
            print("Unsupported chart type.")
            return

        display(fig)

In [214]:
file_dropdown.observe(lambda change: update_and_reload(), names="value")

def update_and_reload():
    global df
    df = load_csv(file_dropdown.value)
    update_dropdowns(df)
    plot_graph(None)

x_dropdown.observe(plot_graph, names="value")
y_dropdown.observe(plot_graph, names="value")
chart_dropdown.observe(plot_graph, names="value")

plot_graph()